# Lab 4 - Linear models

We focus on models in the form 

$$ y \sim \mathrm{Normal}(\alpha+X\beta,\sigma) $$



In [ ]:
from cmdstanpy import CmdStanModel


import arviz as az
import numpy as np
import seaborn as sns
import scipy.stats as stats

import matplotlib.pyplot as plt
import pandas as pd

## Excercise 1 - modelling height of !Kung people

### Normal model - no predictors
We will try to fit $\mathrm{Normal}(\mu,\sigma)$ distribution to height data. Special case of linear model with $\beta=0$.

In [ ]:
_BASE_URL = "https://raw.githubusercontent.com/rmcelreath/rethinking/Experimental/data"
HOWELL_DATASET_PATH = f"{_BASE_URL}/Howell1.csv"
d = pd.read_csv(HOWELL_DATASET_PATH, sep=';', header=0)
d=d[d.age>=18] #just adults 
d.head()

## Task 1. Prior predictive checks

In [ ]:
model_ppc=CmdStanModel(stan_file='/home/lab4/height_1_ppc.stan')

R = 1000
sim=model_ppc.sample(iter_sampling=R,
                     iter_warmup=0,
                     chains=1,
                     fixed_param=True,
                     seed=29042020,refresh=R)



1. Plot histograms of mu, sigma and simulated height.
2. Plot a joint distribution of mu and sigma.
3. Check if samples are consistent with priors.
4. Correct prior parameters to make them reasonably spreaded out. 
5. Check if observed data is possible to obtain using priors.

**Task 1**

In [36]:
ex1 = sim.draws_pd()

,lp__,accept_stat__,alpha,beta,sigma,height[1],height[2],height[3],height[4],height[5],...,height[41],height[42],height[43],height[44],height[45],height[46],height[47],height[48],height[49],height[50]
0,0.0,0.0,-0.203359,0.899524,0.019249,-12.7324,-12.1394,-11.5767,-10.9638,-10.4070,...,10.7209,11.2807,11.9080,12.4963,13.0962,13.6756,14.2370,14.7913,15.3936,15.9847
1,0.0,0.0,-0.203359,0.899524,0.109952,-12.7946,-12.1594,-11.3957,-10.7983,-10.4034,...,10.6061,11.4357,11.7945,12.4419,13.1350,13.9399,14.2293,14.7190,15.3867,15.8852
2,0.0,0.0,-0.203359,0.899524,0.016779,-12.7156,-12.1557,-11.5539,-10.9443,-10.3815,...,10.7219,11.3217,11.8921,12.4611,13.0789,13.6357,14.2309,14.8208,15.4333,15.9677
3,0.0,0.0,-0.203359,0.899524,0.015603,-12.7429,-12.1172,-11.5619,-10.9434,-10.3634,...,10.7175,11.3130,11.9014,12.5084,13.0399,13.6373,14.2247,14.7917,15.4127,15.9701
4,0.0,0.0,-0.203359,0.899524,0.082770,-12.6780,-12.1254,-11.5198,-10.8474,-10.4166,...,10.5846,11.2742,11.9263,12.3418,13.0650,13.5237,14.1411,14.8244,15.4575,15.9300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,-0.203359,0.899524,0.054841,-12.7235,-12.1253,-11.5635,-11.0334,-10.2956,...,10.7555,11.2650,11.9649,12.4930,13.0661,13.6391,14.2949,14.8195,15.4385,15.9877
996,0.0,0.0,-0.203359,0.899524,0.082860,-12.8399,-12.1564,-11.5630,-10.8525,-10.1995,...,10.6971,11.4528,11.9655,12.4361,12.9311,13.7251,14.2318,14.8796,15.5268,16.0814
997,0.0,0.0,-0.203359,0.899524,0.046203,-12.7481,-12.1272,-11.5068,-10.9533,-10.3723,...,10.7436,11.2153,11.8512,12.5030,13.0707,13.6803,14.2243,14.9028,15.4496,15.9588
998,0.0,0.0,-0.203359,0.899524,0.035567,-12.7129,-12.1108,-11.5140,-10.9344,-10.3941,...,10.7589,11.2908,11.8583,12.5422,13.0622,13.6705,14.1503,14.8042,15.3935,15.9730


In [ ]:
mi_sim = ex1["mi"]
sigma_sim = ex1["sigma"]
height_sim = ex1["height_sim"]

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(12, 4))
ax[0].hist(mi_sim, bins=30)
ax[0].set_xlabel("mi")
ax[0].set_ylabel("Frequency")
ax[1].hist(sigma_sim, bins=30)
ax[1].set_xlabel("sigma")
ax[1].set_ylabel("Frequency")
ax[2].hist(height_sim, bins=30)
ax[2].set_xlabel("Simulated Height")
ax[2].set_ylabel("Frequency")
plt.tight_layout()
plt.show()

## Task 2. Model fit and evaluation

In [ ]:
model_1_fit=CmdStanModel(stan_file='height_1_fit.stan')

In [ ]:
fit=model_1_fit.sample(data=dict(N=len(d),
                                   heights=d.height.values),
                         seed=28052020)

***Task 2***

In [ ]:
sns.jointplot(x = "mi", y = "sigma",
              kind = "reg", data = ex1,
              dropna = True)
plt.xlabel("mi")
plt.ylabel("sigma")
plt.show()

In [ ]:
samples = fit.draws()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

axs[0].hist(d.height.values, bins=20, alpha=0.5)
axs[0].set_title('Histogram of Data Heights')

axs[1].hist(height_sim, bins=20, alpha=0.5)
axs[1].set_title('Histogram of Simulated Heights')

plt.show()


1. Plot a joint distribution of fitted mu and sigma.
2. Plot histograms of data and simulated heights and evaluate the quality of model.


## Task 3. Adding predictor to the model - weight

Create column ```c_weight``` in the dataframe containing weights substrated by their mean.


In [16]:
d['c_weight'] = d['weight'] - d['weight'].mean()
data_sim={'N':50, 'weight':np.linspace(d.c_weight.min(),d.c_weight.max())}


***Task 3***

## Task 4. Prior predictive checks
 

In [54]:
model_ppc=CmdStanModel(stan_file='/home/lab4/height_2a_ppc.stan')
R = 1000
sim=model_ppc.sample(data=data_sim, 
                     iter_sampling=R, 
                     iter_warmup=0, 
                     chains=1, 
                     refresh=R,
                     fixed_param=True,
                     seed=29042020)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:CmdStan start processing
chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.


In [56]:
ex2 = sim.draws_pd()

Plot lines for each sampled slope beta and intercept alpha, verify if possible predicted heights are consistent with minimum (0) and maximum (check Wikipedia) heights observed in nature.

## Task 5. Modifying prior

If prior for beta admits negative values, then it makes no sense. Lets change prior to lognormal distribution.


In [ ]:
model_ppc=CmdStanModel(stan_file='height_2b_ppc.stan')

In [ ]:
sim=model_ppc.sample(data=data_sim, 
                     iter_sampling=R, 
                     iter_warmup=0, 
                     chains=1, 
                     refresh=R,
                     fixed_param=True,
                     seed=29042020)

1. Plot lines for each sampled slope beta and intercept alpha, verify if possible predicted heights are consistent with minimum (0) and maximum (check Wikipedia) heights observed in nature.
2. For each simulated weight plot maximum, minimum, and 5, 25, 50, 75, 95 quantiles of simulated weight (all in the same plot). Compare with observed data. Is observed data possible within the prior model?

## Task 6. Fitting and evaluating model


In [ ]:
model_2_fit=CmdStanModel(stan_file='height_2_fit.stan')

1. Create ```data_fit``` dictionary containing data from  ```N``` first rows of dataframe


In [ ]:
fit=model_2_fit.sample(data=data_fit,seed=28052020)


2. Plot lines for each sampled slope beta and intercept alpha. Verify how uncertainity changes with increasing of sample (N)
2. For each value of weight plot mean simulated height along with errorbar of one standard deviation (use ```errorbar``` from matplotlib). Compare with observed data (N points). Is observed data possible within the posterior model? What changes when N increases.


## Task 7. Extending the model

1. Center the weight data (substract mean vaule of weight from all values). Test how model works for such data. What is the interpretation of $\alpha$ in such case?
2. Using centered data modify your model with a second power of weight, select prior for its coefficient using prior predictive checks and after fitting check if model is still good.
3. Try to vectorize the model to avoid necessity of a for loop in the ```model``` block. 